# Klassifikation von Spam Mails und E-Maildaten


in diesem Dokument wird für GPT3.5 tests eine Azure API benötigt, hierzu sollte die Sectrets.toml folgendermaßen aussehen:

[API_TOKEN]

openai_azure = "eigener key"
OPENAI_API_VERSION = "2023-05-15"
OPENAI_API_ENGINE = "name Engine"
OPENAI_API_BASE = "https://spie.openai.azure.com/"

### Imports

In [1]:
import pandas as pd
import openai
import time
import replicate
import os
import tiktoken
import toml

### Tokens und Kosten

In [2]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


In [3]:
def kostenrechner(anzahl_input,anzahl_output):
    if llm == 'gpt-4':
        kosten = ((0.03/1000)*anzahl_input)+((0.06/1000)*anzahl_output)
    elif llm == 'gpt-3.5-turbo':
        kosten = ((0.0015/1000)*anzahl_input)+((0.002/1000)*anzahl_output)
    elif llm == 'replicate/llama70b-v2-chat:e951f18578850b652510200860fc4ea62b3b16fac280f83ff32282f87bbd2e48':        
        kosten = elapsed_time_ges*(0.001400)
    return kosten

### Keys 

In [16]:
llm='gpt-4' #gpt-3.5-turbo, gpt-4, replicate/llama70b-v2-chat:e951f18578850b652510200860fc4ea62b3b16fac280f83ff32282f87bbd2e48

In [17]:
# Laden Sie die .toml-Datei
data = toml.load("../.streamlit/secrets.toml")

In [18]:


# Werte auslesen
if llm!="gpt-3.5-turbo":
    replicate_api_token = data["API_TOKEN"]["replicate_api_token"]
    os.environ['REPLICATE_API_TOKEN'] = replicate_api_token
    openai_api = data["API_TOKEN"]["openai_api"]
    openai.api_key = openai_api
else:
    openai.api_type = "azure"  
    openai.api_key = data["API_TOKEN"]["openai_azure"]
    openai.api_base = data["API_TOKEN"]["OPENAI_API_BASE"]
    openai.api_version = data["API_TOKEN"]["version"]



### promt generierung

In [7]:
def generate_gpt_response(prompt_input):
    messages=[{f"role": "system", "content": context}]
    messages.append({"role": "user", "content": prompt_input})
    if llm == "gpt-3.5-turbo":
        response = openai.ChatCompletion.create(messages=messages, engine=data["API_TOKEN"]["OPENAI_API_ENGINE"])
        full_response = response["choices"][0]["message"]["content"]
    elif llm == 'gpt-4':
        response = openai.ChatCompletion.create(model=llm, messages=messages)
        full_response = response["choices"][0]["message"]["content"]
    else:
        response = replicate.run(llm, 
                           input={"prompt": "You are a helpful assistant. You do not respond as 'User' or pretend to be 'User'. You only respond once as 'Assistant'. "+ context + prompt_input +" Assistant: ",
                                  "temperature":0.1, "top_p":0.9, "max_length":4097, "repetition_penalty":1})
        full_response = ''
        for item in response:
            full_response += item
    return full_response

### Substet

In [8]:
def subset(df,  anzahl, cat_label):
    # Liste zum Speichern der ausgewählten DataFrames
    dfs = []
    # Gruppieren nach 'Kategorie' und nehmen von 5 Stichproben für jede Gruppe
    for name, group in df.groupby(cat_label):
        dfs.append(group.sample(min(len(group), anzahl)))

    # Kombinieren aller gesammelten DataFrames
    con_df = pd.concat(dfs)
    
    return con_df.reset_index(drop=True)

### Test News

In [ ]:
context="I will send you titles of newspaper articles in English now, which you should classify. The categories are: [BUSINESS, ENTERTAINMENT, HEALTH, NATION, SCIENCE, SPORTS, TECHNOLOGY, WORLD]. Always respond with the respective category. Your answer should only be one word. \n The Title starts here:"

In [ ]:
df = pd.read_csv('../Data/labelled_newscatcher_dataset.csv', delimiter=';')
df = df.iloc[:, [0, 4]]


In [ ]:
subset_df = subset(df,20, "topic")


Export DF

In [ ]:
"""subset_df.to_csv('../Data/subset_news_20.csv', index=False)"""

Load DF

In [ ]:
subset_df = pd.read_csv('../Data/subset_news_20.csv')

Start

In [ ]:
"""subset_df = subset_df.iloc[70:90]#subset_df.iloc[:30],  subset_df.iloc[30:60],  subset_df.iloc[60:90], subset_df.iloc[90:120], subset_df.iloc[120:150],  subset_df.iloc[150:180], subset_df.iloc[180:210],  subset_df.iloc[210:240]"""

In [ ]:
TP = 0
Ges = len(subset_df)
ges_len_input = 0
ges_len_responce = 0
elapsed_time_ges = 0
count=0
# Iterieren über die "Text"-Spalte und Drucken jedes Eintrags
for index,text_entry in enumerate(subset_df['title']):
    cat = subset_df["topic"].iloc[index]
    if 'gpt'in llm:
        len_input = num_tokens_from_string(text_entry, llm)
        ges_len_input += len_input
    else:
        ges_len_responce = 0
    #ausführung Modell + Zeitstoppen
    start_time = time.time()
    responce = generate_gpt_response(text_entry)
    end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_ges += elapsed_time
    if 'gpt'in llm:
        len_output_tokens = num_tokens_from_string(responce, llm)
        ges_len_responce += len_output_tokens
    else:
        ges_len_responce = 0
    count+=1
    kosten = kostenrechner(ges_len_input, ges_len_responce)
    if cat == responce:
        TP+=1
    Acc = TP/Ges
    print(f"Class = {cat}, Responce = {responce}, Text = {text_entry}, time: {elapsed_time_ges} cost: {kosten} TP:{TP}")
    
Acc = TP/Ges
kosten = kostenrechner(ges_len_input, ges_len_responce)
print(f"The accuracy of the model is: {Acc} and cost: {kosten} and execution time: {elapsed_time_ges}")


In [ ]:
27/30

### Test Spam

In [9]:


context="Please identify the content that follows the word 'Subject' as either harmless or spam. Reply with 'ham' for harmless and 'spam' for spam. Your answer should only be one word."

In [10]:
"""df_spam = pd.read_csv('../Data/spam_ham_dataset.csv', sep = ",")

df_spam=df_spam[["label","text"]]
"""

'df_spam = pd.read_csv(\'../Data/spam_ham_dataset.csv\', sep = ",")\n\ndf_spam=df_spam[["label","text"]]\n'

In [12]:
"""df_spam_subset = subset(df_spam, 30,"label")"""

'df_spam_subset = subset(df_spam, 30,"label")'

Save DF

In [13]:
"""df_spam_subset.to_csv('../Data/subset_spam_30.csv', index=False)"""

"df_spam_subset.to_csv('../Data/subset_spam_30.csv', index=False)"

Load DF

In [19]:
df_spam_subset = pd.read_csv('../Data/subset_spam_30.csv')

In [20]:
TP = 0
Ges = len(df_spam_subset)
ges_len_input = 0
ges_len_responce = 0
elapsed_time_ges = 0
# Iterieren über die "Text"-Spalte und Drucken jedes Eintrags
for index,text_entry in enumerate(df_spam_subset['text']):
    if 'gpt'in llm:
        len_input = num_tokens_from_string(text_entry, llm)
        ges_len_input += len_input
    else:
        len_input = num_tokens_from_string(text_entry, "gpt-3.5-turbo")
        ges_len_responce = 0
    if len_input>4097:
        text_entry = text_entry[:-int((1-((4097/len_input)-0.05))*len(text_entry))]

    cat = df_spam_subset["label"].iloc[index]
    #ausführung Modell + Zeitstoppen
    start_time = time.time()
    responce = generate_gpt_response(text_entry)
    end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_ges += elapsed_time

    if 'gpt'in llm:
        len_output_tokens = num_tokens_from_string(responce, llm)
        ges_len_responce += len_output_tokens
    else:
        ges_len_responce = 0
    kosten = kostenrechner(ges_len_input, ges_len_responce)
    if cat == responce:
        TP+=1
    print(f"Class = {cat}, Responce = {responce}, Kosten = {kosten}, TP= {TP}")
    
kosten = kostenrechner(ges_len_input, ges_len_responce)
Acc = TP/Ges
print(f"The accuracy of the model is: {Acc} and cost: {kosten} and execution time: {elapsed_time_ges}")


Class = ham, Responce = ham, Kosten = 0.00216, TP= 1
Class = ham, Responce = ham, Kosten = 0.008579999999999999, TP= 2
Class = ham, Responce = ham, Kosten = 0.027419999999999996, TP= 3
Class = ham, Responce = ham, Kosten = 0.038729999999999994, TP= 4
Class = ham, Responce = ham, Kosten = 0.046079999999999996, TP= 5
Class = ham, Responce = ham, Kosten = 0.049859999999999995, TP= 6
Class = ham, Responce = ham, Kosten = 0.050699999999999995, TP= 7
Class = ham, Responce = ham, Kosten = 0.05151, TP= 8
Class = ham, Responce = ham, Kosten = 0.05639999999999999, TP= 9
Class = ham, Responce = ham, Kosten = 0.05697, TP= 10
Class = ham, Responce = ham, Kosten = 0.05927999999999999, TP= 11
Class = ham, Responce = ham, Kosten = 0.07214999999999999, TP= 12
Class = ham, Responce = ham, Kosten = 0.07644, TP= 13
Class = ham, Responce = ham, Kosten = 0.09317999999999999, TP= 14
Class = ham, Responce = ham, Kosten = 0.09449999999999999, TP= 15
Class = ham, Responce = ham, Kosten = 0.11006999999999999, TP

In [ ]:
# Liste zum Speichern der ausgewählten DataFrames
dfs = []

# Gruppieren nach 'Kategorie' und nehmen von 5 Stichproben für jede Gruppe
for name, group in df_spam.groupby('Kategorie'):
    dfs.append(group.sample(min(len(group), 5)))

# Kombinieren aller gesammelten DataFrames
subset_spam_df = pd.concat(dfs)